# DM - Système de décision

## PARTIE 1 : Gurobi

In [71]:
import numpy as np
from gurobipy import Model
from gurobipy.gurobipy import GRB
import random

On génère un dataset qui pourra être résolu par notre solveur. 

Il nous faut donc des valeurs cohérentes de notes par rapport aux coefficients des différentes matières ainsi que des frontières entre les différentes classes (Accepté, Refusé, ...).

In [72]:
def check_class(student, w, frontiers_list, lambdou):
    """
    Retourne la classe d'un étudiant
    :param student: {np.array} Notes d'un élève donné
    :param w: {np.array} Coefficients des matières
    :param frontiers_list: {np.array[]} Frontières des notes pour chaque matière
    :param lambdou: {int} Seuil de majorité
    :return: {int} Retourne la classe d'un étudiant (ex: 0 pour refusé, 1 pour accepté)
    """
    #On part de la classe 0 et on incrémente à chaque fois que la somme des coeffs des matières que l'élève valide pour un niveau donné dépasse lambda
    student_class = 0

    #frontiers_list contient une liste de notes frontières pour chaque matières
    #on transforme pour avoir une liste de frontières des matières pour chaque classe
    frontiers_t = frontiers_list.T

    for frontiers in frontiers_t:
        coeffs_sum = ((student >= frontiers) * w).sum()
        if coeffs_sum >= lambdou:
            student_class += 1
        else:
            break

    return student_class

In [73]:
def convert_dataset(S, y, classes_count):
    """
    :param S: {np.array[]} Notes des élèves
    :param y: {list} Classe des élèves
    :param classes_count: {int} Nombre de classes dans le dataset
    :return: {list(np.array()[]} Liste de classes_count matrices de notes
    """
    X = [[] for i in range(classes_count)]

    for i in range(len(y)):
        student_class = y[i]
        X[student_class].append(list(S[i]))

    return [np.array(matrix) for matrix in X]

In [74]:
def generate_dataset(students_count, subjects_count, classes_count):
    """
    Crée un dataset de notes d'étudiant et de leur classe attribuée
    :param students_count: {int} Nombre d'étudiants
    :param subjects_count: {int} Nombre de matières
    :param classes_count:  {int} Nombre de classes (ex: 2 pour {Accepté, Refusé})
    :return: {list(np.array()[]} Liste de classes_count matrices de notes
    """

    assert students_count > classes_count
    #On génère les coefficients des différentes matières, dont la somme vaut 1
    w_ects = np.random.randint(0, 30, subjects_count)
    w = w_ects / w_ects.sum()
    print("\nLes coeffs sont = ", w)

    #Les frontières entre chaque classe (si 2 classes A et R, seulement 1 frontière)
    frontiers_list = np.zeros((subjects_count, classes_count - 1))
    for i in range(subjects_count):
        frontiers = np.sort(random.sample(list(range(1, 21)), classes_count - 1))
        frontiers_list[i,] = frontiers
    print("\nLes frontières sont = ", frontiers_list)

    #La somme des coeffs minimales à avoir pour être dans une classe donnée
    lambdou = random.uniform(0.5, 1)
    print("\nlambda = ", lambdou)

    y = []
    S = []
    while len(set(y)) != classes_count:
        y = []
        #On génère les notes des élèves (1 ligne par élève)
        S = np.random.randint(0, 21, (students_count, subjects_count))
        print("\nLes notes des élèves :")
        print(S)

        #On regarde à quelle classe est associé chaque élève en fonction de ses notes

        for student in S:
            student_class = check_class(student, w, frontiers_list, lambdou)
            y.append(student_class)

    print("\nLes classes des élèves :")
    print(y)

    #On veut retourner une matrice par classe 
    #Si 2 classes A et R : A contient les notes des élèves qui sont admis et R les notes des élèves refusés
    X = convert_dataset(S, y, classes_count)

    return X


In [75]:
X = generate_dataset(10, 4, 3)


Les coeffs sont =  [0.49122807 0.31578947 0.         0.19298246]

Les frontières sont =  [[ 1.  3.]
 [ 1.  6.]
 [12. 20.]
 [ 3.  8.]]

lambda =  0.6665411771241101

Les notes des élèves :
[[ 2  4 19  3]
 [ 1  4  9 14]
 [14 17  2  1]
 [ 1  7 20  5]
 [19 15  1  6]
 [15 11 16 10]
 [16  3 14 13]
 [20 18 15  5]
 [16  3 15 20]
 [10 19 18 15]]

Les notes des élèves :
[[ 3  9 15  5]
 [10 13 20 11]
 [14 10 11 17]
 [13  6 19 16]
 [ 2 13  6 11]
 [18  7  6 17]
 [15  3  9 18]
 [ 4  4  6 13]
 [ 4  9  4  9]
 [ 5  5  9 14]]

Les notes des élèves :
[[ 4 10 10  6]
 [20  3 20 13]
 [ 6  0 14 16]
 [18  2  1 11]
 [10 13  3 13]
 [ 2 11 16  1]
 [16  5 11 18]
 [ 9 17 15 20]
 [10  5 17  2]
 [ 9 20 13 19]]

Les notes des élèves :
[[18 11 19 20]
 [10 19 18 15]
 [ 8  0 18 19]
 [11  4  7  8]
 [19 20  2 16]
 [ 8 10 19 11]
 [ 5  5  1 16]
 [ 7 16 14  4]
 [ 2 15  7  9]
 [ 4 20  5 12]]

Les notes des élèves :
[[ 5 19 20 17]
 [10  6  0  3]
 [ 0 12  4  9]
 [ 6 10 19 16]
 [ 8  5 11 11]
 [19 15 14  5]
 [ 7 17 17 16]
 [13  

In [76]:
def solve(X, students_count, subjects_count, classes_count, verbose=0):
    """
    :param X: {list(np.array()[]} Liste de classes_count matrices de notes
    :param students_count: {int} Nombre d'étudiants
    :param subjects_count: {int} Nombre de matières
    :param classes_count:  {int} Nombre de classes (ex: 2 pour {Accepté, Refusé})
    :param verbose:
    :return:
    """
    R, *L, A = X

    # Instanciation du modèle
    m = Model("PL modeling")

    #------------------VARIABLES------------------

    # Vecteur des coefficients des différentes matières
    w = m.addMVar(shape=subjects_count, lb=0, ub=1, name="w")

    c_accepted = m.addMVar(shape=(len(A), subjects_count), lb=0, ub=1, name="c_accepted")
    c_refused = m.addMVar(shape=(len(R), subjects_count), lb=0, ub=1, name="c_refused")

    delta_accepted = m.addMVar(shape=(students_count, subjects_count), vtype=GRB.INTEGER, lb=0, ub=1,
                               name="delta_accepted")
    delta_refused = m.addMVar(shape=(students_count, subjects_count), vtype=GRB.INTEGER, lb=0, ub=1, name="delta_refused")

    sigma_accepted = m.addMVar(shape=len(A), name="x", vtype=GRB.CONTINUOUS)
    sigma_refused = m.addMVar(shape=len(R), name="y", vtype=GRB.CONTINUOUS)

    lambd = m.addVar(name="lambda", lb=0.5, ub=1)

    b = m.addMVar(shape=subjects_count, name="b", lb=0, ub=20)

    alpha = m.addVar(name="alpha")

    m.update()

    epsilon = 0.01

    M = 150
    
    
    #------------------CONTRAINTES------------------

    # Validation constraint with sigma
    for index, student_point in enumerate(c_accepted):
        m.addConstr(sum(student_point) - sigma_accepted[index] + epsilon == lambd)

    # Refusal constraint with sigma
    for index, student_point in enumerate(c_refused):
        m.addConstr(sum(student_point) == lambd - sigma_refused[index])

    # Alpha is inferior than all sigmas
    for offset in (*sigma_accepted, *sigma_refused):
        m.addConstr(alpha <= offset)

    # # cij is inferior than wi
    # for matrix in (c_accepted ,c_refused):
    #     for student in matrix:
    #         for index, point in enumerate(student):
    #             m.addConstr(point <= w[index])

    accepted_tuple = (c_accepted, delta_accepted, A)
    refused_tuple = (c_refused, delta_refused, R)

    for c, delta, grades in (accepted_tuple, refused_tuple):
        for i in range(c.shape[0]):
            for j in range(c.shape[1]):
                m.addConstr(c[i, j] <= w[j])
                m.addConstr(c[i, j] <= delta[i, j])
                m.addConstr(c[i, j] >= delta[i, j] - 1 + w[j])
                m.addConstr(M * delta[i, j] + epsilon >= grades[i, j] - b[j])
                m.addConstr(M * (delta[i, j] - 1) <= grades[i, j] - b[j])

    m.addConstr(sum(w) == 1)

    m.update()

    if verbose == 0:
        m.params.outputflag = 0

    #Fonction objectif
    m.setObjective(alpha, GRB.MAXIMIZE)
    m.update()

    m.optimize()
    print("====== Model solved =====")
    print(f"alpha = {alpha.X}")
    print(f"lambda = {lambd.X}")
    print(f"b = {b.X}")

    print("\nLes coefficients de notre solution optimale sont :")
    print(f"__________________")

    print(f"| Matière  |coeff|")

    for index, coef in enumerate(w.X):
        print(f"|matière {index + 1} | {coef.round(2)} |")
    print(f"__________________")

    return A, R, w.X, lambd.X, b.X


### Exemple 1 : 

20 élèves, 5 matières et 2 classes (Accepté ou Refusé) 

In [77]:
X = generate_dataset(20, 5, 2)


Les coeffs sont =  [0.10769231 0.2        0.29230769 0.03076923 0.36923077]

Les frontières sont =  [[11.]
 [15.]
 [ 7.]
 [ 9.]
 [ 5.]]

lambda =  0.6848556873743621

Les notes des élèves :
[[17  6  4 20 12]
 [ 4 10  0 20 19]
 [ 5 18 12 19 16]
 [ 8 20  0 10 18]
 [20 16 14  2  0]
 [19  2  3 19 11]
 [ 2 11  3  1 12]
 [15  3  2  7  0]
 [ 6 11  4 14 14]
 [16  0  4 15  9]
 [17  9 14 10  5]
 [14 15 13 12  8]
 [ 4 16  0 12  1]
 [17  1  4 17  0]
 [10 16  3  3  1]
 [16 10 18 14  6]
 [16  6  2 13  4]
 [ 9 20 15 17 14]
 [ 7  2 13 14 18]
 [ 3 10  4 14  1]]

Les classes des élèves :
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0]


In [78]:
accepted, refused, weigths, lambd, b = solve(X, 20, 5, 2, verbose=0)

====== Model solved =====
alpha = 0.255
lambda = 0.755
b = [ 2.    1.   12.    6.99  0.99]

Les coefficients de notre solution optimale sont :
__________________
| Matière  |coeff|
|matière 1 | 0.0 |
|matière 2 | 0.0 |
|matière 3 | 0.5 |
|matière 4 | 0.5 |
|matière 5 | 0.0 |
__________________


In [79]:
print((accepted >= b) @ weigths.T >= lambd)

[ True  True  True  True  True  True]


In [80]:
print((refused >= b) @ weigths.T >= lambd)

[False False False False False False False False False False False False
 False False]


**Conclusion :**

Avec 2 classes (Accepté, Refusé), notre MRSort fonctionne et nous renvoie une solution faisable pour notre dataset.